The goal of this notebook is to explore some of the functionality needed for a first layer lambda function.  
This lambda function is mainly for security purposes and scrubbing PII from the data before doing analysis on the main version.  

This notebook begins to ...
* check for zipped file and extracts 
* open JSON and checks schema of several of the inner dictionaries 
* delete PII fields in User dictioanry


Remember to add security and environment controls to next part (for downloading and uploading to s3/ other parts of the drive for the save)  

Also add the security to the runtime  

DO some file play work as well

In [55]:
import json 
import os 
import pprint as pp
import uuid

In [59]:
uuid.uuid4()

UUID('f71e8829-4c10-4956-b386-ca14d7b11909')

In [2]:
os.getcwd()

'/home/owner/Documents/Github_projects/Tinder_analysis'

In [3]:
zip_path = '/home/owner/Documents/Github_projects/Tinder_analysis/Data/myData.zip'

In [51]:
!ls $new_temp_path

data.json  index.html  your_photos


In [4]:
zip_path[-4:] == '.zip'

True

In [5]:
from zipfile import ZipFile

In [6]:
assert(os.path.isfile(zip_path))

In [68]:
str(uuid.uuid1())

'1b593f38-358f-11e9-bde6-6c2995106085'

In [61]:
new_temp_path = "/tmp/zip_extracts"+str(uuid.uuid1())
if not os.path.isdir(new_temp_path):
    os.mkdir(new_temp_path)

In [62]:
new_temp_path

'/tmp/zip_extracts4797344e-d3b9-460b-ac40-54971c219e11'

In [33]:
# opening the zip file in READ mode 
with ZipFile(zip_path, 'r') as zip: 
    # printing all the contents of the zip file 
    zip.printdir() 
  
    # extracting all the files 
    print('Extracting all the files now...') 
    zip.extractall(new_temp_path) 
    print('Done!')

File Name                                             Modified             Size
your_photos/photo_0.jpg                        2018-12-05 04:00:00        85073
your_photos/photo_1.jpg                        2018-12-05 04:00:00       123535
your_photos/photo_2.jpg                        2018-12-05 04:00:00        71596
your_photos/photo_3.jpg                        2018-12-05 04:00:00        74983
your_photos/photo_4.jpg                        2018-12-05 04:00:00        32387
index.html                                     2018-12-05 04:00:00       293763
data.json                                      2018-12-05 04:00:00       233030
Extracting all the files now...
Done!


In [34]:
path = new_temp_path+'/data.json'

In [35]:
assert(os.path.isfile(path))

In [36]:
with open(path, 'rb') as hnd:
    data = json.load(hnd)

In [37]:
top_level_keys = {'Messages', 'Photos', 'Places', 'Purchases', 'Spotify', 'Usage', 'User'}
data_keys = set(data.keys())

In [38]:
# Check schema 
top_level_keys.intersection(data_keys) == top_level_keys

True

In [39]:
# No dups
len(data_keys) == len(data.keys())

True

In [40]:
# Nothing new in data keys 
len(top_level_keys.difference(data_keys)) == 0

True

In [41]:
len(data_keys.difference(top_level_keys)) == 0

True

In [42]:
def schema_check(input_list, control_set):
    
    errors = []
    input_set = set(input_list)
    
    # No dups in input 
    if len(input_set) != len(input_list): errors.append("Duplicates in input list")
    
    # Everything in control list should be covered 
    if len(control_set.difference(input_set)) != 0: errors.append("Extra in control-- "+ str(control_set.difference(input_set)))
    
    # Everything in data should be covered
    if len(input_set.difference(control_set)) != 0: errors.append("Extra in control-- "+ str(input_set.difference(control_set)))
    
    
    return(errors)

In [69]:
schema_check(data.keys(), top_level_keys)

[]

In [43]:
schema = {}
for k1,v in data.items():
    if type(v) == dict:
        schema[k1] = v.keys()

In [44]:
list(schema.keys())

['Places', 'Purchases', 'Spotify', 'Usage', 'User']

In [45]:
pp.pprint(schema)

{'Places': dict_keys(['recentPlaces', 'blockedPlaces', 'analytics']),
 'Purchases': dict_keys(['subscription', 'consumable', 'boost_tracking', 'super_like_tracking']),
 'Spotify': dict_keys(['spotify_connected', 'spotify_theme_track']),
 'Usage': dict_keys(['app_opens', 'swipes_likes', 'swipes_passes', 'matches', 'messages_sent', 'messages_received']),
 'User': dict_keys(['active_time', 'age_filter_max', 'age_filter_min', 'bio', 'birth_date', 'connection_count', 'create_date', 'education', 'email', 'full_name', 'gender', 'gender_filter', 'interested_in', 'interests', 'ip_address', 'is_traveling', 'jobs', 'name', 'pos', 'schools', 'travel_location_info', 'travel_pos', 'phone_id', 'college'])}


In [46]:
controlled_schema = {'Places': {'recentPlaces', 'blockedPlaces', 'analytics'},
'Purchases': {'subscription', 'consumable', 'boost_tracking', 'super_like_tracking'},
'Spotify': {'spotify_connected', 'spotify_theme_track'}, 
'Usage': {'app_opens', 'swipes_likes', 'swipes_passes', 'matches', 'messages_sent', 'messages_received'}, 
'User':{'active_time', 'age_filter_max', 'age_filter_min', 'bio', 'birth_date',
                 'connection_count', 'create_date',  'education', 'gender', 'gender_filter',
                 'jobs', 'schools'}}

In [47]:
errors= {}
for k in data_keys:
    if k in controlled_schema.keys():
        errors[k] = schema_check(data[k].keys(), controlled_schema[k])

In [48]:
errors

{'Places': [],
 'Spotify': [],
 'Usage': [],
 'User': ["Extra in control-- {'phone_id', 'interested_in', 'travel_location_info', 'name', 'full_name', 'college', 'travel_pos', 'email', 'ip_address', 'is_traveling', 'interests', 'pos'}"],
 'Purchases': []}

In [49]:
user = data['User']

In [50]:
user.keys()

dict_keys(['active_time', 'age_filter_max', 'age_filter_min', 'bio', 'birth_date', 'connection_count', 'create_date', 'education', 'email', 'full_name', 'gender', 'gender_filter', 'interested_in', 'interests', 'ip_address', 'is_traveling', 'jobs', 'name', 'pos', 'schools', 'travel_location_info', 'travel_pos', 'phone_id', 'college'])

In [25]:
user

{'active_time': '2018-12-03T11:51:24.738Z',
 'age_filter_max': 28,
 'age_filter_min': 22,
 'bio': 'New to Brooklyn \nInterests include salsa dancing and data science 💃\n\nHere for a good time not a long time',
 'birth_date': '1993-09-14T00:00:00.000Z',
 'connection_count': 1376,
 'create_date': '2014-03-22T20:49:10.255Z',
 'education': 'Has high school and/or college education',
 'email': 'themalchemistway@gmail.com',
 'full_name': 'Malcolm Taylor',
 'gender': 'M',
 'gender_filter': 'F',
 'interested_in': 'F',
 'interests': [{'id': '227871678136099',
   'created_time': '2018-11-23T19:25:07+0000',
   'name': 'Sam Torres'},
  {'id': '70926594298',
   'created_time': '2018-09-15T05:03:12+0000',
   'name': 'ALDO Shoes'},
  {'id': '528365127526143',
   'created_time': '2017-12-22T02:20:39+0000',
   'name': 'Elena Argudin'},
  {'id': '112573976071113',
   'created_time': '2017-08-05T12:20:33+0000',
   'name': 'SB'},
  {'id': '291059770920854',
   'created_time': '2014-08-09T03:16:51+0000',
 

## Things to drop


- email, full_name, travel_pos, ip_address, phone_id
- travel_location_info -> route, street_number, lat, lon

In [26]:
user2 = user.copy()

In [27]:
user_drops = ['email', 'full_name', 'travel_pos', 'ip_address', 'phone_id']

In [28]:
for key in user_drops:
    if key in user2:
        try:
            del user2[key]
        except:
            errors.append("Failed to drop key: "+ str(key))
            pass

In [29]:
user_travel_drops = ['route', 'street_number', 'lat', 'lon']

In [30]:
for key in user_travel_drops:
    if key in user2:
        try:
            del user2['travel_location_info'][key]
        except:
            errors.append("Failed to drop key: "+ str(key))
            pass 

## End with serialization 

In [72]:
with open("demo_dump.json", "w") as hnd:
    json.dump(data, hnd)